In [1]:
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
import s3fs
import warnings 
import math

Open U data

In [2]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-u.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
du = xr.open_zarr(store=store, consolidated=True)
du = du.rename({'depthu' : 'depth'})
du 

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, depth: 75, time_counter: 72, x: 1580, y: 1801)
Coordinates:
  * depth                 (depth) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    nav_lat               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered         (time_counter) object dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter          (time_counter) object 2000-01-03 12:00:00 ... 2000-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    depthu_bounds         (time_counter, depth, axis_nbounds) float32 dask.array<chunksize=(1, 75, 2), meta=np.ndarray>
    sozotaux              (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) object dask.array<chunksize=(72, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) object dask.array<chunksize=(72, 2), meta=np.ndarray>
    utau_atmoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    utau_iceoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    uwspd10               (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vozocrtx              (time_counter, depth, y, x) float32 dask.array<chunksize=(1, 75, 1801, 1580), meta=np.ndarray>
Attributes:
    CASE:              REF08
    CONFIG:            CREG12.L75
    Conventions:       CF-1.5
    description:       ocean U grid variables
    name:              /scratch/cnt0028/lpo7420/talandic/CREG12.L75-REF08-XIO...
    output_frequency:  5d
    production:        An IPSL model
    start_date:        19790101
    timeStamp:         2020-May-14 08:39:48 CEST
    title:             ocean U grid variables

Open V data 

In [3]:
#open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path = 's3://velocity-v.zarr/'
remote_files = fsg.glob(data_path)

store = s3fs.S3Map(root=data_path, s3=fsg, check=False)
dv = xr.open_zarr(store=store, consolidated=True)
dv = dv.rename({'depthv' : 'depth'})
dv 

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, depth: 75, time_counter: 73, x: 1580, y: 1801)
Coordinates:
  * depth                 (depth) float32 0.5058 1.556 ... 5.698e+03 5.902e+03
    nav_lat               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon               (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered         (time_counter) object dask.array<chunksize=(73,), meta=np.ndarray>
  * time_counter          (time_counter) object 2000-01-03 12:00:00 ... 2000-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    depthv_bounds         (time_counter, depth, axis_nbounds) float32 dask.array<chunksize=(1, 75, 2), meta=np.ndarray>
    sometauy              (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    time_centered_bounds  (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    time_counter_bounds   (time_counter, axis_nbounds) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    vomecrty              (time_counter, depth, y, x) float32 dask.array<chunksize=(1, 75, 1801, 1580), meta=np.ndarray>
    vtau_atmoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vtau_iceoce           (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
    vwspd10               (time_counter, y, x) float32 dask.array<chunksize=(1, 1801, 1580), meta=np.ndarray>
Attributes:
    CASE:              REF08
    CONFIG:            CREG12.L75
    Conventions:       CF-1.5
    description:       ocean V grid variables
    name:              /scratch/cnt0028/lpo7420/talandic/CREG12.L75-REF08-XIO...
    output_frequency:  5d
    production:        An IPSL model
    start_date:        19790101
    timeStamp:         2020-May-14 08:39:49 CEST
    title:             ocean V grid variables

In [4]:
u = du.vozocrtx
v = dv.vomecrty

Ubar and Vbar

In [5]:
ubar = u[0,:,:,:]
vbar = v[0,:,:,:]
N = 0

for i in range(1,len(u.time_counter)):
    ubar += u[i,:,:,:]
    vbar += v[i,:,:,:]
    N += 1
    
ubar = ubar/N
vbar = vbar/N

In [6]:
ubar.load()

<xarray.DataArray 'vozocrtx' (depth: 75, y: 1801, x: 1580)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        (y, x) float32 25.58 25.58 25.58 25.58 ... 55.09 55.06 55.04
    nav_lon        (y, x) float32 -93.63 -93.54 -93.46 ... 102.1 102.0 101.9
    time_centered  object 2000-01-03 12:00:00
    time_counter   object 2000-01-03 12:00:00
Dimensions without coordinates: y, x

In [7]:
vbar.load()

<xarray.DataArray 'vomecrty' (depth: 75, y: 1801, x: 1580)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        (y, x) float32 25.62 25.62 25.62 25.62 ... 55.09 55.06 55.04
    nav_lon        (y, x) float32 -93.67 -93.58 -93.5 ... 102.1 102.1 102.0
    time_centered  object 2000-01-03 12:00:00
    time_counter   object 2000-01-03 12:00:00
Dimensions without coordinates: y, x

Uvar and Vvar

In [11]:
uvar = (u[0,:,:,:])**2
vvar = (v[0,:,:,:])**2
N = 0

for i in range(1,len(u.time_counter)):
    ubar += (u[i,:,:,:])**2
    vbar += (v[i,:,:,:])**2
    N += 1
    
uvar = uvar/N - ubar**2
vvar = vvar/N - vbar**2

UVbar

In [9]:
uvbar = (u[0,:,:,:]*v[0,:,:,:])
N = 0 

for i in range(1,len(u.time_counter)):
    uvbar += (u[i,:,:,:]*v[i,:,:,:])
    N += 1
uvbar = uvbar/N - ubar*vbar 

In [16]:
uvar

<xarray.DataArray 'vozocrtx' (depth: 75, y: 1801, x: 1580)>
dask.array<sub, shape=(75, 1801, 1580), dtype=float32, chunksize=(75, 1801, 1580), chunktype=numpy.ndarray>
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered  object dask.array<chunksize=(), meta=np.ndarray>
    time_counter   object 2000-01-03 12:00:00
Dimensions without coordinates: y, x

Merge to one dataset 

In [17]:
covmatrix_data = xr.merge([ubar.to_dataset(name = 'ubar'),
                         vbar.to_dataset(name = 'vbar'),
                         uvar.to_dataset(name = 'uvar'),
                         vvar.to_dataset(name = 'vvar'),
                         uvbar.to_dataset(name = 'uvbar')], compat='override')

Save as zarr

In [18]:
covmatrix_data

<xarray.Dataset>
Dimensions:        (depth: 75, x: 1580, y: 1801)
Coordinates:
  * depth          (depth) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
    nav_lat        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(1801, 1580), meta=np.ndarray>
    time_centered  object dask.array<chunksize=(), meta=np.ndarray>
    time_counter   object 2000-01-03 12:00:00
Dimensions without coordinates: x, y
Data variables:
    ubar           (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    vbar           (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    uvar           (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    vvar           (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>
    uvbar          (depth, y, x) float32 dask.array<chunksize=(75, 1801, 1580), meta=np.ndarray>

In [ ]:
#covmatrix_data.to_zarr('s3://data/covmatrix_data.zarr', consolidated=True)

In [19]:
# choosing a point 
x = 1020 
y = 1540 

In [ ]:
A = xr.DataArray([[covmatrix_data.uvar[:,y,x],covmatrix_data.uvbar[:,y,x]],
                  [covmatrix_data.uvbar[:,y,x],covmatrix_data.vvar[:,y,x]]], dims=["i", "j", "depth"])
A

In [ ]:
'Function to calculate the eigenvalues, eigenvector an the featuring matrix for given level'

def eigen_func(covmatrix, level):

    #eigenvectors and eigenvalues
    value_list = np.zeros((75,2))
    vector_list = np.zeros((75,2,2))

    for i in range(len(covmatrix.depth)):
        values, vectors = np.linalg.eig(covmatrix[:,:,i])
        print('vectors:  ', vectors)
        print('values:  ',values)
        value_list[i] = values
        vector_list[i] = vectors               

    #print(vector_list.shape)
    #print(value_list.shape)

    #Feature vectors chosen from the eigenvectors I want to keep with the highest corresponding eigenvalue
    fm_ = np.array([vector_list[level,0,:],vector_list[level,1,:]])
    #print(fm[:,0])
    #print(fm[:,1])
    
    return fm_

In [ ]:
n = 0

fm = eigen_func(A,n)

#plot the data 
fig = plt.figure(figsize=(7, 7))
fig.suptitle('Velocity')
plt.xlabel('U')
plt.ylabel('V')

rådata.isel(depth=0, time_counter=slice(0,72)).plot.scatter(x='vozocrtx', y='vomecrty', s=6)

plt.plot([0,fm[0,0]*np.sqrt(value_list[0,0])],[0,fm[1,0]*np.sqrt(value_list[0,0])],'r') #PC1 
plt.plot([0,fm[0,0]*np.sqrt(value_list[0,0])*(-1)],[0,fm[1,0]*np.sqrt(value_list[0,0])*(-1)],'r')
plt.plot([0,fm[0,1]*np.sqrt(value_list[1,0])],[0,fm[1,1]*np.sqrt(value_list[1,0])],'k') #PC2
plt.plot([0,fm[0,1]*np.sqrt(value_list[1,0])*(-1)],[0,fm[1,1]*np.sqrt(value_list[1,0])*(-1)],'k')